# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from pickle import dump


nltk.download(['punkt', 'wordnet', 'stopwords'])

[nltk_data] Downloading package punkt to /home/lucas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/lucas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/lucas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)
df = df.dropna()
X = df['message']
y = df.drop(columns=['id', 'message', 'genre'])

In [3]:
df.shape

(25789, 39)

### 2. Write a tokenization function to process your text data

In [11]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()        
    stop_words = list(stopwords.words('english')) # About 150 stopwords   

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        if clean_tok.isalpha() and clean_tok not in stop_words:                        
            clean_tokens.append(clean_tok)

    return clean_tokens    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([            
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),    
    ('clf', RandomForestClassifier())
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
import sklearn
sklearn.__version__

'0.24.2'

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fa604fd43a0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier())])

In [15]:
with open('model.pkl', 'wb') as f:
    dump(pipeline, f)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
y_pred = pipeline.predict(X_test)
res = y_pred == y_test
print(classification_report(y_test, y_pred, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.76      0.96      0.85      4864
               request       0.52      0.07      0.13      1098
                 offer       0.00      0.00      0.00        28
           aid_related       0.45      0.14      0.21      2626
          medical_help       0.00      0.00      0.00       468
      medical_products       0.17      0.00      0.01       318
     search_and_rescue       0.00      0.00      0.00       178
              security       0.00      0.00      0.00       108
              military       0.00      0.00      0.00       192
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       372
                  food       0.24      0.01      0.01       685
               shelter       0.13      0.00      0.01       584
              clothing       0.25      0.01      0.02        89
                 money       0.00      

/home/lucas/anaconda3/envs/banqi/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lucas/anaconda3/envs/banqi/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lucas/anaconda3/envs/banqi/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/luc

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7fa604fd43a0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', RandomForestClassifier())],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7fa604fd43a0>),
 'tfidf': TfidfTransformer(),
 'clf': RandomForestClassifier(),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf': True,
 'clf__bootstrap': True,
 'clf__ccp_alpha': 0.0,


In [19]:
parameters = {
    # 'clf__estimator__criterion': [
    #     'mse', 'friedman_mse',
    #     'mae', 'poisson',
    # ],
    # 'clf__estimator__splitter': [
    #     'best', 'random'
    # ],
    'clf__criterion': ['gini', 'entropy']

}
cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7fa604fd43a0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf', RandomForestClassifier())]),
             param_grid={'clf__criterion': ['gini', 'entropy']})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [20]:
y_pred = cv.predict(X_test)
res = y_pred == y_test
print(classification_report(y_test, y_pred, target_names=y_test.columns))

                        precision    recall  f1-score   support

               related       0.76      0.97      0.85      4864
               request       0.47      0.06      0.11      1098
                 offer       0.00      0.00      0.00        28
           aid_related       0.45      0.09      0.15      2626
          medical_help       0.00      0.00      0.00       468
      medical_products       0.20      0.00      0.01       318
     search_and_rescue       0.00      0.00      0.00       178
              security       0.00      0.00      0.00       108
              military       0.00      0.00      0.00       192
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       372
                  food       0.25      0.00      0.01       685
               shelter       0.14      0.00      0.01       584
              clothing       0.25      0.01      0.02        89
                 money       0.00      

/home/lucas/anaconda3/envs/banqi/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lucas/anaconda3/envs/banqi/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lucas/anaconda3/envs/banqi/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/luc

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [21]:
with open('model.pkl', 'wb') as f:
    dump(pipeline, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.